<img src="NotebookAddons/blackboard-banner.png" width="100%" />

# Tile Data Stack to MGRS (Military Grid Reference System)

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

**Alex Lewandowski; University of Alaska Fairbanks**

This notebook subsets a tiff stack into MGRS tiles

* Takes a directory of geotiffs
* Creates directories named by MGRS tile, filled with MGRS tile subsets
* Allows for saving or deletion of original geotiffs

### Important Note about JupyterHub

**Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.**


In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 0. Import Relevant Python Packages

We will use the following scientific library:
- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.

**mport the necesssary libraries and modules:**

In [ ]:
%%capture
from pathlib import Path
from math import ceil
from tqdm import tqdm
from typing import Dict, List, Tuple, Union

import mgrs
from mgrs.core import MGRSError
import numpy as np
from osgeo import gdal
from pyproj import CRS, Transformer

from ipyfilechooser import FileChooser

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

**Enter the path to the directory holding your tiffs:**

In [ ]:
fc = FileChooser('/home/jovyan/notebooks')
display(fc)

**Find the paths of all geotiffs in the data directory:**

In [ ]:
data_dir = Path(fc.selected_path)
tiff_paths = list(data_dir.glob('*.tif*'))

## 1. Subset to MGRS Tiles

**Create some necessary lookup tables**


In [ ]:
# 'I' and 'O' are omitted from MGRS grids to avoid confusion with '0' and '1' when printed on paper maps.
# This makes char math difficult but we can use lookup tables to deal with it.

GNUM = {
    'A': 1, 'B': 2, 'C': 3,
    'D': 4, 'E': 5, 'F': 6,
    'G': 7, 'H': 8, 'J': 9,
    'K': 10, 'L': 11, 'M': 12,
    'N': 13, 'P': 14, 'Q': 15,
    'R': 16, 'S': 17, 'T': 18,
    'U': 19, 'V': 20, 'W': 21,
    'X': 22, 'Y': 23, 'Z': 24
}

GLET = {v: k for (k,v) in GNUM.items()}

# Valid 100k rows follow a regular pattern for odd and even UTMs 
# (represented by the zero-padded, two-digit number at the start of an MGRS tile)

ZONE_ROWS = {
    'odd': {
        'X': ['V', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P'],
        'W': ['M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V'],
        'V': ['C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L'],
        'U': ['P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A', 'B', 'C'],
        'T': ['E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P'],
        'S': ['R', 'S', 'T', 'U', 'V', 'A', 'B', 'C', 'D', 'E'],
        'R': ['G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R'],
        'Q': ['T', 'U', 'V', 'A', 'B', 'C', 'D', 'E', 'F', 'G'],
        'P': ['J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T'],
        'N': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'],
        'M': ['M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V'],
        'L': ['C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M'],
        'K': ['P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A', 'B', 'C'],
        'J': ['E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P'],
        'H': ['R', 'S', 'T', 'U', 'V', 'A', 'B', 'C', 'D', 'E'],
        'G': ['G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R'],
        'F': ['T', 'U', 'V', 'A', 'B', 'C', 'D', 'E', 'F', 'G'],        
        'E': ['K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T'],
        'D': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K'],
        'C': ['M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A'],        
    },
    'even': {
        'X': ['E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U'],
        'W': ['S', 'T', 'U', 'V', 'A', 'B', 'C', 'D', 'E'],
        'V': ['H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R'],
        'U': ['U', 'V', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
        'T': ['K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U'],
        'S': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K'],
        'R': ['M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A'],
        'Q': ['C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M'],
        'P': ['P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A', 'B', 'C'],
        'N': ['F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P'],
        'M': ['S', 'T', 'U', 'V', 'A', 'B', 'C', 'D', 'E'],
        'L': ['H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S'],
        'K': ['U', 'V', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
        'J': ['K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U'],
        'H': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K'],
        'G': ['M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A'],
        'F': ['C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M'],        
        'E': ['Q', 'R', 'S', 'T', 'U', 'V', 'A', 'B', 'C'],
        'D': ['F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q'],
        'C': ['S', 'T', 'U', 'V', 'A', 'B', 'C', 'D', 'E', 'F'],                
    }
}

**Write functions to do the work of identifying all MGRS tiles touching a geotiff**

In [ ]:
def utm_to_latlon(easting: float, northing: float, epsg: str) -> Tuple[float, float]:
    """
    converts coordinates from any EPSG into lat-lon (EPSG: 4326)
    """
    transformer = Transformer.from_crs(f"epsg:{epsg}", "epsg:4326", always_xy=True)
    return transformer.transform(easting, northing)

def get_epsg(path: Union[str, Path]) -> str:
    """
    returns the EPSG of a geotiff
    """
    info = gdal.Info(str(path), format='json')
    return info['coordinateSystem']['wkt'].split('ID')[-1].split(',')[1][0:-2]

def get_corners(path: Union[str, Path]) -> Dict[str, float]:
    """
    returns the corner coordinate metadata of a geotiff
    """
    return gdal.Info(str(path), format='json')['cornerCoordinates']


def get_mgrs_corners(path: Union[str, Path]) -> Dict[str, str]:
    """
    returns the 100k MGRS tiles containing each corner of a geotiff
    """
    epsg = get_epsg(path)
    corners = get_corners(path)

    ul = utm_to_latlon(corners['upperLeft'][0], corners['upperLeft'][1], epsg)
    lr = utm_to_latlon(corners['lowerRight'][0], corners['lowerRight'][1], epsg)

    corners = dict()
    m = mgrs.MGRS()
    corners.update({'ul': m.toMGRS(ul[1], ul[0], MGRSPrecision=0)})
    corners.update({'ll': m.toMGRS(lr[1], ul[0], MGRSPrecision=0)})
    corners.update({'ur': m.toMGRS(ul[1], lr[0], MGRSPrecision=0)})
    corners.update({'lr': m.toMGRS(lr[1], lr[0], MGRSPrecision=0)})
    return corners

def valid_utm_columns(utm: int) -> List[str]:
    """
    utm: the zero-padded, two-digit number at the start of an MGRS tile
    returns a list of valid column letters for a given utm
    """
    if utm in range(1, 61, 3):
        return ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    if utm in range(2, 61, 3):
        return ['J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R']
    if utm in range(3, 61, 3):
        return ['S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

def get_zones(mgrs_corners: Dict[str, str]) -> List[str]:
    """
    returns a list of all MGRS zones in area bounded by mgrs_corners
    MGRS zone: the uppercase letter at index 2 of an MGRS tile string
    """
    zones = list(range(GNUM[mgrs_corners['lr'][2]], GNUM[mgrs_corners['ur'][2]]+1))
    return [GLET[i] for i in zones] 

def get_utms(mgrs_corners: Dict[str, str]) -> List[str]:
    """
    returns a list of all UTMs (as zero-padded strings) in area bounded by mgrs_corners
    UTM: the zero-padded, two-digit number at the start of an MGRS tile
    """
    utms = list(range(int(mgrs_corners['ul'][:2]), int(mgrs_corners['ur'][:2])+1))
    if len(utms) == 0:
        utms = list(range(1, int(mgrs_corners['ur'][:2])+1))
        utms += list(range(int(mgrs_corners['ul'][:2]), 61))
    return [f"{i:02d}" for i in utms]   

    
def get_utm_zones(mgrs_corners: Dict[str, str]) -> List[str]:
    """
    returns a list of a valid UTM zones in area bounded by mgrs_corners
    UTM zones: indices [0:3] of 100k MGRS grid string
    """
    utms = get_utms(mgrs_corners)
    zones = get_zones(mgrs_corners)
    
    utm_zones = list()
    for utm in utms:
        for zone in zones:
            utm_zones.append(f"{utm}{zone}")
    return utm_zones

def get_utm_zones_cols(utm_zones: List[str], mgrs_corners: Dict[str, str]) -> List[str]:
    """
    returns a list of all valid UTM Zones and 100k MGRS columns in area bounded by mgrs_corners:
    indices [0:4] of 100k MGRS grid strings
    
    UTM zones: list of indices [0:3] of 100k MGRS grid string
    100k MGRS column: index 3 of 100k MGRS grid string
    """
    utm_zones_cols = list()
    for g in utm_zones:
        valid_cols = valid_utm_columns(int(g[:2]))
        if g[:2] == mgrs_corners['ur'][:2]:
            east_col = mgrs_corners['ur'][3]
            index = valid_cols.index(east_col)
            for col in valid_cols[:index+1]:
                utm_zones_cols.append(f"{g}{col}")
        elif g[:2] == mgrs_corners['ul'][:2]:
            west_col = mgrs_corners['ul'][3]
            index = valid_cols.index(west_col)
            for col in valid_cols[index:]:
                utm_zones_cols.append(f"{g}{col}")
        else:
            for col in valid_cols:
                utm_zones_cols.append(f"{g}{col}")
    return utm_zones_cols              
                
def get_utm_zones_cols_rows(utm_zones_cols: List[str], mgrs_corners: Dict[str, str]) -> List[str]:
    """
    returns a list of all 100k MGRS grid tiles in area bounded by mgrs_corners
    utm_zones_cols: list of indices [0:4] of 100k MGRS grid strings
    """
    utm_zones_cols_rows = list()
    eastern_utm_even = int(mgrs_corners['ur'][:2]) % 2 == 0
    
    for col in utm_zones_cols:
        col_utm = int(col[:2])
        col_utm_even = col_utm % 2 == 0
        
        n_eastern_rows_even = ZONE_ROWS['even'][mgrs_corners['ur'][2]]
        s_eastern_rows_even = ZONE_ROWS['even'][mgrs_corners['lr'][2]]
        n_eastern_rows_odd = ZONE_ROWS['odd'][mgrs_corners['ur'][2]]
        s_eastern_rows_odd = ZONE_ROWS['odd'][mgrs_corners['lr'][2]]
        
        if col_utm_even:
            valid_rows = ZONE_ROWS['even'][col[2]]
            if eastern_utm_even:
                index_north = n_eastern_rows_even.index(mgrs_corners['ur'][4])
                index_south = s_eastern_rows_even.index(mgrs_corners['lr'][4])
            else:
                index_north = n_eastern_rows_odd.index(mgrs_corners['ur'][4])
                index_south = s_eastern_rows_odd.index(mgrs_corners['lr'][4])
        else:
            valid_rows = ZONE_ROWS['odd'][col[2]]
            if not eastern_utm_even:
                index_north = n_eastern_rows_odd.index(mgrs_corners['ur'][4])
                index_south = s_eastern_rows_odd.index(mgrs_corners['lr'][4])
            else:
                index_north = n_eastern_rows_even.index(mgrs_corners['ur'][4])
                index_south = s_eastern_rows_even.index(mgrs_corners['lr'][4])
            
        if col[2] == mgrs_corners['ur'][2] and col[2] == mgrs_corners['lr'][2]:
            valid_rows = valid_rows[index_south:index_north+1]
        elif col[2] == mgrs_corners['ur'][2]:
            valid_rows = valid_rows[:index_north+1]
        elif col[2] == mgrs_corners['lr'][2]:
            valid_rows = valid_rows[index_south:]
        for row in valid_rows:
            utm_zones_cols_rows.append(f"{col}{row}")
    return utm_zones_cols_rows

**Subset the tiffs by MGRS tile, saving them into directories named `<data_directory>/<UTM tile><MGRS tile>/`**

In [ ]:
tile_side_len = 100000 #for 100km tiles
resolution = int(gdal.Info(str(tiff_paths[0]), format='json')['geoTransform'][1])
padding  = int(((ceil(tile_side_len / resolution) * resolution) - tile_side_len))

reproj_paths = list()
reproj_dirs = list()
for tif in tqdm(tiff_paths):
    print(f"\ngeotiff: {tif}")
    
    tif_epsg = get_epsg(tif)
    gdal.Warp(str(tif), str(tif),
                          dstSRS=f'EPSG:{tif_epsg}',
                          xRes=resolution, yRes=resolution, targetAlignedPixels=True)
    
    mgrs_corners = get_mgrs_corners(tif)
#     print(f"mgrs_corners: {mgrs_corners}")
    
    utm_zones = get_utm_zones(mgrs_corners)
#     print(f"utm_zones: {utm_zones}")
    
    utm_zones_cols = get_utm_zones_cols(utm_zones, mgrs_corners)
#     print(f"utm_zones_cols: {utm_zones_cols}")
    
    tiles = get_utm_zones_cols_rows(utm_zones_cols, mgrs_corners)
    print(f"MGRS Tiles: {tiles}")
    
    for tile in tiles:
        tile_dir = data_dir/f"{tile}"
        if not tile_dir.exists():
            tile_dir.mkdir()
        dest = tile_dir/f"{tif.stem}_{tile}.tif"
        
        m = mgrs.MGRS()
        print(tile)
        utm = m.MGRSToUTM(tile)

        ulx = utm[2] - padding
        uly = utm[3] + tile_side_len + padding 
        lrx = utm[2] + tile_side_len + padding
        lry = utm[3] - padding
        
        
        if utm[1] == 'N':
            hemisphere = 'north'
        else:
            hemisphere = 'south'
            
        
        crs = CRS.from_string(f'+proj=utm +zone={utm[0]} +{hemisphere}')
        tile_epsg = crs.to_authority()[1]
        if tile_epsg != tif_epsg:
            reproj_path = tif.parent/f"{tile_epsg}_reproj/{tif.stem}_{tile_epsg}.tif"
            reproj_paths.append(reproj_path)
            reproj_dirs.append(reproj_path.parent)
            if not reproj_path.exists():
                if not reproj_path.parent.exists():
                    reproj_path.parent.mkdir()
                gdal.Warp(str(reproj_path), str(tif),
                          srcSRS=f'EPSG:{tif_epsg}', dstSRS=f'EPSG:{tile_epsg}',
                          xRes=resolution, yRes=resolution, targetAlignedPixels=True)
            
            dest = tile_dir/f"{tif.stem}_{tile}.tif"
            
            gdal.Translate(destName=str(dest), srcDS=str(reproj_path), projWin=[ulx, uly, lrx, lry])
        else:
            gdal.Translate(destName=str(dest), srcDS=str(tif), projWin=[ulx, uly, lrx, lry])
            pass
        
for p in set(reproj_paths):
    p.unlink()
for p in set(reproj_dirs):
    p.rmdir()

**Cleanup temporary files and tiles containing no data**

No-data filled tiles can occur if the portion of the original geotiff in an MGRS tile contained no-data to start with.

*An improvement to this notebook would be to check for no-data prior to creating a tile, instead of deleting it later.*

In [ ]:
print("Do you wish to save or delete the original geotiffs?")
orig_tiffs = asfn.select_parameter(['save', 'delete'], '')
display(orig_tiffs)

In [ ]:
tile_paths = list(data_dir.glob('*/*.tif*'))

removed = []
for f in tile_paths:
    raster = gdal.Open(str(f))
    band = raster.ReadAsArray()
    if np.count_nonzero(band) < 1:
        f.unlink()
        removed.append(f)

if len(removed) == 0:
    print("No tiles were removed due to no-data.")
else:
    print(f"{len(removed)} no-data tiles removed:")
    for f in removed:
        print(f)     

removed = set([p.parent for p in removed])
for p in removed:
    print(p.iterdir())
    if len(list(p.iterdir())) == 0:
        p.rmdir()
        
if orig_tiffs.value == 'delete':
    for tif in tiff_paths:
        tif.unlink()

**Print the paths to the directories holding your MGRS tiles:**

In [ ]:
for d in [i for i in list(data_dir.iterdir()) if not i.name.startswith('.')]:
    print(d)

*MGRS_Tile_Data_Stack - Version 1.1.0 - November 2021*

*Changes:*

- *asf_notebook.py -> opensarlab-lib*
- *url-widget*
- *remove unused imports*